In [ ]:
import tensorflow_datasets as tfds

# Load the dataset
dataset, info = tfds.load('ted_hrlr_translate/fr_to_pt', with_info=True, as_supervised=True)

# Print dataset information
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/43873 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1131 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.C49FN5_1.0.0/ted_hrlr_translate-val…

Generating test examples...:   0%|          | 0/1494 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/incomplete.C49FN5_1.0.0/ted_hrlr_translate-tes…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/fr_to_pt/1.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='ted_hrlr_translate',
    full_name='ted_hrlr_translate/fr_to_pt/1.0.0',
    description="""
    Data sets derived from TED talk transcripts for comparing similar language pairs
    where one is high resource and the other is low resource.
    """,
    config_description="""
    Translation dataset from fr to pt in plain text.
    """,
    homepage='https://github.com/neulab/word-embeddings-for-nmt',
    data_dir=PosixGPath('/tmp/tmp9dvldo_3tfds'),
    file_format=tfrecord,
    download_size=124.94 MiB,
    dataset_size=9.74 MiB,
    features=Translation({
        'fr': Text(shape=(), dtype=string),
        'pt': Text(shape=(), dtype=string),
    }),
    supervised_keys=('fr', 'pt'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=1494, num_shards=1>,
        'train': <